# Faculdades

> Check faculdades_clean.ipynb for state of data cleaning for Faculdades

Para criar uma versão `html` sem o código fonte, fazer, na directoria "notebooks":

    jupyter nbconvert faculdades.ipynb --to html --no-input

## Setup

In [1]:
from timelink.mhk.utilities import get_engine, get_dbnames, get_connection_string
from sqlalchemy import engine, text, MetaData
print('Available databases:')
print(get_dbnames())
%load_ext sql
%config SqlMagic.displaycon=False

Available databases:
['alunos', 'angelica', 'api_tests', 'china', 'china_coimbra', 'dyncoopnet', 'eiras', 'familias', 'glopes', 'ilhavo', 'ilhavo2', 'ilhavo_contrib', 'ilhavo_editor', 'ilhavo_server', 'ilhavo_testes', 'lousa', 'lousa3', 'lousa4', 'mhk', 'notaveis', 'obidos', 'oguest', 'rhv', 'santaclara', 'santajusta', 'sister', 'soure', 'soure_edit', 'soure_editor', 'soure_mfg', 'soure_public', 'soure_server', 'testes', 'toliveira', 'toliveira_reviewed', 'ucprosop']


## Connect to database

### Choose the database

In [2]:
db = 'ucprosop'
connection_string = get_connection_string(db)
%sql $connection_string
engine = get_engine(db)

## Map MHK database

In [3]:
from sqlalchemy import MetaData,Table, Column, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
mhk = MetaData()
Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()
entities_table = Table('entities', mhk, autoload_with=engine)
attr_table = Table('attributes', mhk, autoload_with=engine)
nattr_table = Table('nattributes', mhk, autoload_with=engine)
person_table = Table('persons', mhk, autoload_with=engine)

## Stats for faculdade

In [4]:
import pandas
from sqlalchemy import create_engine

# we need this because pandas 1.3.2 not compatible with recent sqlalchemy
legacy_engine = create_engine(connection_string,future=False)

fac_df = pandas.read_sql(
    f"select distinct entity,\
             the_value as faculdade,\
             SUBSTRING(the_date,1,4) as year \
             from attributes \
             where the_type = 'faculdade' \
                   and the_date <> '0000-00-00' \
        order by year",
    con=legacy_engine, # this is necessary for compatibility of read_sql with sqlalchemy 2.0
)
fac_df = fac_df[fac_df['faculdade'].isin(['Artes','Teologia','Cânones','Leis','Medicina','Direito','Matemática','Filosofia','?'])]
fac_df = fac_df.drop_duplicates(subset=['entity','year', 'faculdade'], keep='first')
#print("Faculdades: remoção de múltiplos valores por aluno, mantendo o primeiro")
#fac_df.info()
print("Exemplo do resultado:")
fac_df[:10]

Exemplo do resultado:


,entity,faculdade,year
0,242228,Artes,1256
1,196431,Artes,1503
2,141858,Cânones,1536
3,172287,Artes,1536
4,128342,Cânones,1537
5,129439,Leis,1537
6,130313,Cânones,1537
7,131972,Cânones,1537
8,132974,Leis,1537
9,135969,Leis,1537


In [5]:
import numpy as np
pandas.set_option('display.max_rows', None)

table = pandas.pivot_table(fac_df,index=['year'], columns=['faculdade'],aggfunc=np.count_nonzero, fill_value=0)
table

entity                                                           \
faculdade      ? Artes Cânones Direito Filosofia Leis Matemática Medicina   
year                                                                        
1256           0     1       0       0         0    0          0        0   
1503           0     1       0       0         0    0          0        0   
1536           0     1       1       0         0    0          0        0   
1537           1     2      69       1         0   78          0        7   
1538           0     0      66       0         0   87          0        2   
1539           0     1      17       0         0   17          0        0   
1540           0     8     106       0        12  148          0       14   
1541           0    13       7       1         0   20          0        4   
1542           0    10       3       0         1    5          0        3   
1543           0     9       5       0         0    9          0        1   
1544           0     3       2       0         0    6          0        1   
1545           0     1       2       0         0    2          0        0   
1546           0     3       5       0         0    3          0        0   
1547           0     0       9       1         0    6          0        2   
1548           0    13      17       0         0   11          0        8   
1549           0    18      44       1         3   22          0       10   
1550           0    57      70       0         0   27          0       13   
1551           0    27      43       0         0    9          0        7   
1552           1    28      42       2         0   32          0        9   
1553           0    24      36       1         0   28          0        4   
1554           0    26      27       1         0   21          0        8   
1555           0    41      34       0         0   19          0        2   
1556           0    24      34       1         1   32          0        1   
1557           0    46      39       0         0   12          0        2   
1558           0    39      45       0         1   27          0        7   
1559           0    12      45       0         0   17          0        6   
1560           0    20      48       0         0   19          0        5   
1561           0    19      35       0         0   13          0        2   
1562           0    22      53       0         0   28          0        2   
1563           0     1      64       0         0   19          0        2   
1564           0    23      60       0         0   22          0        5   
1565           0    23      35       0         0   22          0        4   
1566           0    21      39       1         0   14          0        7   
1567           0    19      64       0         0   26          0        5   
1568           0    21      50       0         0   22          0        9   
1569           0    16      36       0         1    7          0        8   
1570           0    14      52       0         0   31          0        9   
1571           0    18      48       1         0   17          0       13   
1572           0     9      67       1         0   38          0        5   
1573           1    11     153       0         0   73          0       17   
1574           0    34      57       0         0   43          0       12   
1575           0     8      37       0         0   32          0        6   
1576           0    15      58       0         0   43          0       12   
1577           0     1     157       0         0   85          0       22   
1578           2     6     114       0         0   53          0       15   
1579           2     1     121       0         0   50          0       15   
1580           0     7      58       0         0   28          0       14   
1581           0     6      86       0         0   34          0       10   
1582           0     3     113       0         0   40          0       13   
1583  